In [45]:
# Import
import requests
import pandas as pd
import numpy as np

import re

import urllib
import bs4
from urllib import request

import sys
!{sys.executable} -m pip install geopy
import geopy
from geopy.geocoders import Nominatim
from geopy import distance

from datetime import date
from datetime import timedelta

# ML
import seaborn as sns
import matplotlib.pyplot as plt
import base64
import string
import re
!pip install nltk
import nltk

from collections import Counter
from nltk.corpus import stopwords
from time import time
#from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS as stopwords
from sklearn.metrics import log_loss
import matplotlib.pyplot as plt
!pip install pywaffle
from pywaffle import Waffle

from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation

nltk.download('punkt')
nltk.download('stopwords')
stop_words = set(stopwords.words('french'))

[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
# Adds the part about the user right here

class User:
    def __init__(self, address, date, price_max = 42):
        geolocator = Nominatim(user_agent="CulturalEvents")
        location = geolocator.geocode(address)
        self.latlon = (location.latitude, location.longitude)
        self.date = date
        
Naclanga = User("27 Boulevard Thomas Gobert, Palaiseau", date.today())

In [4]:
# Data Extraction

# Query to extract a specific frame of time. We have to change it for the user desired date
query = "date_start < " + str(Naclanga.date + timedelta(days=7)) + " AND date_start > "  + str(Naclanga.date)
url = "https://data.iledefrance.fr/api/records/1.0/search/?dataset=evenements-publics-cibul&q=&rows=100&q=(" + query + ")&sort=date_start"
req = requests.get(url)

df = pd.json_normalize(req.json()['records'])

important_keys = ["fields.title", "fields.latlon", "fields.date_start", "fields.tags", 'fields.title', "fields.pricing_info", "fields.description",'fields.link', 'fields.free_text', 'fields.address', 'fields.department','fields.placename', 'fields.region', 'fields.date_end', 'fields.lang', 'geometry.type', 'geometry.coordinates', 'fields.city_district']
df = df[important_keys]

dict_important_keys = {elt : elt.replace("fields.", "") for elt in important_keys}
df.rename(columns = dict_important_keys, inplace=True)

df_to_treat = df.copy()
df_to_treat

,title,latlon,date_start,tags,title,pricing_info,description,link,free_text,address,department,placename,region,date_end,lang,geometry.type,geometry.coordinates,city_district
0,2E2M • VOYAGER / MÉFANO 1,"[48.878667, 2.320671]",2022-01-04,"concert,création musicale,musique contemporain...",2E2M • VOYAGER / MÉFANO 1,Entrée libre dans la limite des places disponi...,Dans le cadre des 50 ans de 2e2m,http://openagenda.com/event/2e2m-voyager-mefan...,En 2022 2e2m fête ses 50 ans. L’occasion pour...,"14 rue de Madrid, 75008 Paris",NaN,CRR de Paris - Auditorium Marcel Landowski,Île-de-France,2022-01-04,fr,Point,"[2.320671, 48.878667]",Quartier de l'Europe
1,Exposition : Le chemin des incertitudes,"[48.824766, 2.27503]",2022-01-04,"Walid Stefanesco,clavim",Exposition : Le chemin des incertitudes,NaN,Calligraphies numériques de Walid Stefanesco.,http://openagenda.com/event/le-chemin-des-ince...,"Habitant d’Issy, Walid Stefanesco s’exprime pa...","60, rue du Général Leclerc 92130 Issy-Les-Moul...",Hauts-de-Seine,ESPACE ANDREE CHEDID,Île-de-France,2022-01-13,fr,Point,"[2.27503, 48.824766]",Quartier Centre-Ville / Corentin Celton / Les ...
2,Massiwa + L’Expat / chorégraphie Salim Mzé Ha...,"[48.609589, 2.312712]",2022-01-04,DANSE,Massiwa + L’Expat / chorégraphie Salim Mzé Ha...,"tarif orange (6, 12 ou 18€)",DANSE URBAINE ET DANSE DU MONDE,http://openagenda.com/event/massiwa-lexpat-cho...,**Chorégraphie Salim Mzé Hamadi Moissi**\n\nMê...,Rue Henri Douard 91220 BRETIGNY SUR ORGE,Essonne,THEATRE BRETIGNY,Île-de-France,2022-01-04,fr,Point,"[2.312712, 48.609589]",NaN
3,"Formation : Enjeux, bonnes pratiques et labell...","[48.856697, 2.351462]",2022-01-04,"Numérique Responsable,LNR,Label NR,Institut du...","Formation : Enjeux, bonnes pratiques et labell...",1500€ HT - Formation de 3 jours soit 21 heures...,"4, 5 et 11 janvier 2022 - Formation en visioc...",http://openagenda.com/event/formation-enjeux-b...,"""De retour d'une formation riche de trois jour...",Paris,Paris,En visioconférence,Île-de-France,2022-01-05,fr,Point,"[2.351462, 48.856697]",NaN
4,Avant-première surprise : Film coup de cœur de...,"[48.883948, 2.440764]",2022-01-04,NaN,Avant-première surprise : Film coup de cœur de...,NaN,Film coup de coeur de l'Association Française ...,http://openagenda.com/event/avant-premiere-sur...,NaN,"Place Carnot, Romainville, France",Seine-Saint-Denis,Cinéma Le Trianon,Île-de-France,2022-01-04,fr,Point,"[2.440764, 48.883948]",NaN
5,HISTOIRE DES ATTITUDES DEVANT LA MORT DEPUIS L...,"[48.800506, 2.131408]",2022-01-04,NaN,HISTOIRE DES ATTITUDES DEVANT LA MORT DEPUIS L...,Sur inscription - Tarif B (Cycle de 6 conféren...,La réinvention occidentale de la crémation au ...,http://openagenda.com/event/histoire-des-attit...,"PAR GUILLAUME CUCHET, AGRÉGÉ D’HISTOIRE, PROFE...","6 impasse des Gendarmes, entrée B 78000 Versai...",Yvelines,Université Ouverte de Versailles,Île-de-France,2022-01-04,fr,Point,"[2.131408, 48.800506]",Chantiers
6,George Dandin - Molière/Lully,"[48.804425, 2.120285]",2022-01-04,NaN,George Dandin - Molière/Lully,Réservation sur https://www.chateauversailles-...,Comédie mêlée d'une pastorale dans laquelle Mo...,http://openagenda.com/event/george-dandin-moli...,Pour mettre en abîme cette fable à la fois dou...,Château de Versailles,Yvelines,Opéra Royal,Île-de-France,2022-01-08,fr,Point,"[2.120285, 48.804425]",Notre-Dame
7,Hommage à Didier LOCKWOOD avec David &amp; Tho...,"[48.859865, 2.347728]",2022-01-04,NaN,Hommage à Didier LOCKWOOD avec David &amp; Tho...,tarif plein : 30€ ; tarif réduit : 20€,Il est le musicien qui est à l’origine du lieu...,http://openagenda.com/event/hommage-a-didier-l...,"Thomas Enhco, David Enhco, Benoit Sourisse, An...",60 rue des lombards,NaN,Sunset Sunside,Île-de-France,2022-01-04,fr,Point,"[2.347728, 48.859865]",Quartier Les Halles
8,Atelier généalogie,"[48.829127, 2.317717]",2022-01-04,"généalogie,recherche,ancestres",Atelier généalogie,Réservé aux adhérents de l'association (15€ l'...,Rechercher ses ancêtres e

In [187]:
def extract_price_regex(x):
    L = []
    for k in re.findall("\S+\s?€", str(x)):
        try:
            a = float(k.replace("€", "").replace(",", "."))
        except ValueError:
            # It doesn't work
            a = float("NaN")
        L.append(a)
    return np.mean(L)

df_to_treat['pricing_info'] = df['pricing_info'].apply(lambda x:extract_price_regex(x))

In [188]:
df_to_treat

,title,latlon,date_start,tags,title,pricing_info,description,link,free_text,address,department,placename,region,date_end,lang,geometry.type,geometry.coordinates,city_district
0,Colloque AVISA,"[48.631891, 2.431379]",2022-12-09,NaN,Colloque AVISA,NaN,Écrire l’histoire du harcèlement sexuel sur la...,http://openagenda.com/event/colloque-avisa,Colloque en hybride,2 rue Facteur Cheval - Evry-Courcouronnes,Essonne,Université d'Évry - UFR SHS,Île-de-France,2022-12-10,fr,Point,"[2.431379, 48.631891]",Évry
1,Le Carnaval des Animaux Sud-américains,"[48.889308, 2.393804]",2022-11-20,"Ensemble ALMAVIVA,Carnaval des animaux sud-amé...",Le Carnaval des Animaux Sud-américains,NaN,"Bienvenue à Rio de Janeiro, qui accueille le c...",http://openagenda.com/event/le-carnaval-des-an...,Doté depuis son enfance à Rio du pouvoir de co...,"221 avenue Jean-Jaurès, 75019 Paris",Paris,La Philharmonie de Paris,Île-de-France,2022-11-20,fr,Point,"[2.393804, 48.889308]",NaN
2,Logique des passions (titre provisoire),"[48.800061, 2.403721]",2022-11-17,"théâtre,immersion,installation,numérique,vidéo...",Logique des passions (titre provisoire),NaN,Le Diptyque Collectif,http://openagenda.com/event/logique-des-passio...,Ce spectacle invite le public à vivre ou reviv...,"13, rue Pierre Sémard. 94400 Vitry-sur-Seine",Val-de-Marne,Nouveau Gare au Théâtre,Île-de-France,2022-11-19,fr,Point,"[2.403721, 48.800061]",Port à l'Anglais
3,Expoprotection,"[48.832528, 2.287004]",2022-11-15,"expoprotection,santé et sécurité au travail,sé...",Expoprotection,NaN,Le salon de la prévention et de la gestion des...,http://openagenda.com/event/expoprotection_640,"Tous les deux ans à Paris, Expoprotection rass...","Place de la Porte de Versailles, 75015 Paris",Paris,Porte de Versailles,Île-de-France,2022-11-17,fr,Point,"[2.287004, 48.832528]",NaN
4,EquipHotel,"[48.832488, 2.287499]",2022-11-06,"hôtellerie,restauration,collectivités",EquipHotel,NaN,Hotel &amp; Restaurant Business Place,http://openagenda.com/event/equiphotel_359,"Du 6 au 10 novembre 2022, ne manquez pas le re...",1 place de la Porte de Versailles,Paris,Paris expo Porte de Versailles,Île-de-France,2022-11-10,fr,Point,"[2.287499, 48.832488]",15e Arrondissement
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,Sirba Octet &amp; Balalaïka,"[48.765358, 2.268554]",2022-06-07,NaN,Sirba Octet &amp; Balalaïka,15.0,SIRBALALAÏKA,http://openagenda.com/event/test-sirba-octet-a...,"Voir un concert du Sirba Octet, c’est comme pa...","254, avenue de la Division Leclerc 92290 châte...",Hauts-de-Seine,L'Azimut - Théâtre La Piscine,Île-de-France,2022-06-07,fr,Point,"[2.268554, 48.765358]",NaN
96,L'Aparté vous accueille ce samedi,"[48.824529, 2.274148]",2022-06-04,"CLAVIM,Aparté,Maison Verte",L'Aparté vous accueille ce samedi,NaN,Lieu d’accueil pour les bébés et les enfants d...,http://openagenda.com/event/laparte-vous-accue...,L’**Aparté** vous accueille sans rendez-vous c...,"60 rue du Général Leclerc, 92130 Issy-les-Moul...",Hauts-de-Seine,L'Aparté,Île-de-France,2022-06-04,fr,Point,"[2.274148, 48.824529]",NaN
97,Erstkommunion,"[48.870856, 2.285033]",2022-06-04,NaN,Erstkommunion,NaN,Festgottesdienst der Erstkommunion in St. Hono...,http://openagenda.com/event/erstkommunion-4029827,NaN,"66 bis avenue Raymond-Poincaré, 75116 Paris",Paris,St Honoré d'Eylau,Île-de-France,2022-06-04,fr,Point,"[2.285033, 48.870856]",Paris 16e Arrondissement
98,Solennité de Pentecôte,"[48.855518, 2.354704]",2022-06-04,NaN,Solennité de Pentecôte,NaN,Célébrer le don de l'Esprit Saint avec les Fra...,http://openagenda.com/event/solennite-de-pente...,* **Samedi 4 juin 2022**\n\n12h30 : messe du...,"13, rue des Barres, 75004 Paris",Paris,"Église Saint-Gervais,Paris",Île-de-France,2022-06-05,fr,Point,"[2.354704, 48.855518]",Paris 4e Arrondissement


In [43]:
df_to_treat['distance'] = df['latlon'].apply(lambda x: distance.distance(x, Naclanga.latlon))
df_to_treat

,title,latlon,date_start,tags,title,pricing_info,description,link,free_text,address,department,placename,region,date_end,lang,geometry.type,geometry.coordinates,city_district,distance
0,Colloque AVISA,"[48.631891, 2.431379]",2022-12-09,NaN,Colloque AVISA,Inscription,Écrire l’histoire du harcèlement sexuel sur la...,http://openagenda.com/event/colloque-avisa,Colloque en hybride,2 rue Facteur Cheval - Evry-Courcouronnes,Essonne,Université d'Évry - UFR SHS,Île-de-France,2022-12-10,fr,Point,"[2.431379, 48.631891]",Évry,19.36843905054484 km
1,Le Carnaval des Animaux Sud-américains,"[48.889308, 2.393804]",2022-11-20,"Ensemble ALMAVIVA,Carnaval des animaux sud-amé...",Le Carnaval des Animaux Sud-américains,NaN,"Bienvenue à Rio de Janeiro, qui accueille le c...",http://openagenda.com/event/le-carnaval-des-an...,Doté depuis son enfance à Rio du pouvoir de co...,"221 avenue Jean-Jaurès, 75019 Paris",Paris,La Philharmonie de Paris,Île-de-France,2022-11-20,fr,Point,"[2.393804, 48.889308]",NaN,24.29276779242778 km
2,Logique des passions (titre provisoire),"[48.800061, 2.403721]",2022-11-17,"théâtre,immersion,installation,numérique,vidéo...",Logique des passions (titre provisoire),"sur réservations , tarifs 13 euros TP et 10 eu...",Le Diptyque Collectif,http://openagenda.com/event/logique-des-passio...,Ce spectacle invite le public à vivre ou reviv...,"13, rue Pierre Sémard. 94400 Vitry-sur-Seine",Val-de-Marne,Nouveau Gare au Théâtre,Île-de-France,2022-11-19,fr,Point,"[2.403721, 48.800061]",Port à l'Anglais,17.92231496774433 km
3,Expoprotection,"[48.832528, 2.287004]",2022-11-15,"expoprotection,santé et sécurité au travail,sé...",Expoprotection,Demandez votre badge gratuit,Le salon de la prévention et de la gestion des...,http://openagenda.com/event/expoprotection_640,"Tous les deux ans à Paris, Expoprotection rass...","Place de la Porte de Versailles, 75015 Paris",Paris,Porte de Versailles,Île-de-France,2022-11-17,fr,Point,"[2.287004, 48.832528]",NaN,14.797863061932679 km
4,EquipHotel,"[48.832488, 2.287499]",2022-11-06,"hôtellerie,restauration,collectivités",EquipHotel,NaN,Hotel &amp; Restaurant Business Place,http://openagenda.com/event/equiphotel_359,"Du 6 au 10 novembre 2022, ne manquez pas le re...",1 place de la Porte de Versailles,Paris,Paris expo Porte de Versailles,Île-de-France,2022-11-10,fr,Point,"[2.287499, 48.832488]",15e Arrondissement,14.809879786818652 km
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,Sirba Octet &amp; Balalaïka,"[48.765358, 2.268554]",2022-06-07,NaN,Sirba Octet &amp; Balalaïka,Tarif plein : 20€ / réduit : 15€ / super rédui...,SIRBALALAÏKA,http://openagenda.com/event/test-sirba-octet-a...,"Voir un concert du Sirba Octet, c’est comme pa...","254, avenue de la Division Leclerc 92290 châte...",Hauts-de-Seine,L'Azimut - Théâtre La Piscine,Île-de-France,2022-06-07,fr,Point,"[2.268554, 48.765358]",NaN,7.771512371269672 km
96,L'Aparté vous accueille ce samedi,"[48.824529, 2.274148]",2022-06-04,"CLAVIM,Aparté,Maison Verte",L'Aparté vous accueille ce samedi,Entrée libre sans rendez-vous,Lieu d’accueil pour les bébés et les enfants d...,http://openagenda.com/event/laparte-vous-accue...,L’**Aparté** vous accueille sans rendez-vous c...,"60 rue du Général Leclerc, 92130 Issy-les-Moul...",Hauts-de-Seine,L'Aparté,Île-de-France,2022-06-04,fr,Point,"[2.274148, 48.824529]",NaN,13.591714003213303 km
97,Erstkommunion,"[48.870856, 2.285033]",2022-06-04,NaN,Erstkommunion,NaN,Festgottesdienst der Erstkommunion in St. Hono...,http://openagenda.com/event/erstkommunion-4029827,NaN,"66 bis avenue Raymond-Poincaré, 75116 Paris",Paris,St Honoré d'Eylau,Île-de-France,2022-06-04,fr,Point,"[2.285033, 48.870856]",Paris 16e Arrondissement,18.670708459014364 km
98,Solennité de Pentecôte,"[48.855518, 2.354704]",2022-06-04,NaN,Solennité de Pentecôte,NaN,Célébrer le don de l'Esprit Saint avec les Fra...,http://openagenda.com/event/solennite-de-pente...,* **Samedi 4 juin 2022**\n\n12h30 : messe du...,"13, ru

In [189]:
# Date : we extract the hours of opening

In [190]:
def get_hours_regex(text):
    return str(text.select('span.hour')[0]).replace('<span class="hour">', "").replace("</span>", "")
    
def get_text(link):
    request_text = request.urlopen(link).read()
    page = bs4.BeautifulSoup(request_text)
    return(page)
    
def get_timespan(df_link):
    L = []
    for link in df_link:
        try:
            text = get_text(link)
            hours = get_hours_regex(text)
            L.append(hours)
        except (urllib.error.HTTPError, IndexError): #403 error case, if the Event is not correctly setup and if the event is not accessible
            L.append(float("NaN"))
        
    return L

df_to_treat['opening_hours'] = get_timespan(df['link']) # Time consuming, so we should get all information at once to avoid doing too much webscraping

# Should we differency ponctual events (movies, plays, sports) and long running events (expositions) ?
# -> We can probably do it judging the timespan of the events in terms of days and hours, but is it useful ?

In [191]:
df_to_treat

,title,latlon,date_start,tags,title,pricing_info,description,link,free_text,address,department,placename,region,date_end,lang,geometry.type,geometry.coordinates,city_district,opening_hours
0,Colloque AVISA,"[48.631891, 2.431379]",2022-12-09,NaN,Colloque AVISA,NaN,Écrire l’histoire du harcèlement sexuel sur la...,http://openagenda.com/event/colloque-avisa,Colloque en hybride,2 rue Facteur Cheval - Evry-Courcouronnes,Essonne,Université d'Évry - UFR SHS,Île-de-France,2022-12-10,fr,Point,"[2.431379, 48.631891]",Évry,09:00 - 17:00
1,Le Carnaval des Animaux Sud-américains,"[48.889308, 2.393804]",2022-11-20,"Ensemble ALMAVIVA,Carnaval des animaux sud-amé...",Le Carnaval des Animaux Sud-américains,NaN,"Bienvenue à Rio de Janeiro, qui accueille le c...",http://openagenda.com/event/le-carnaval-des-an...,Doté depuis son enfance à Rio du pouvoir de co...,"221 avenue Jean-Jaurès, 75019 Paris",Paris,La Philharmonie de Paris,Île-de-France,2022-11-20,fr,Point,"[2.393804, 48.889308]",NaN,11:00 - 12:00
2,Logique des passions (titre provisoire),"[48.800061, 2.403721]",2022-11-17,"théâtre,immersion,installation,numérique,vidéo...",Logique des passions (titre provisoire),NaN,Le Diptyque Collectif,http://openagenda.com/event/logique-des-passio...,Ce spectacle invite le public à vivre ou reviv...,"13, rue Pierre Sémard. 94400 Vitry-sur-Seine",Val-de-Marne,Nouveau Gare au Théâtre,Île-de-France,2022-11-19,fr,Point,"[2.403721, 48.800061]",Port à l'Anglais,20:00 - 21:00
3,Expoprotection,"[48.832528, 2.287004]",2022-11-15,"expoprotection,santé et sécurité au travail,sé...",Expoprotection,NaN,Le salon de la prévention et de la gestion des...,http://openagenda.com/event/expoprotection_640,"Tous les deux ans à Paris, Expoprotection rass...","Place de la Porte de Versailles, 75015 Paris",Paris,Porte de Versailles,Île-de-France,2022-11-17,fr,Point,"[2.287004, 48.832528]",NaN,09:00 - 18:00
4,EquipHotel,"[48.832488, 2.287499]",2022-11-06,"hôtellerie,restauration,collectivités",EquipHotel,NaN,Hotel &amp; Restaurant Business Place,http://openagenda.com/event/equiphotel_359,"Du 6 au 10 novembre 2022, ne manquez pas le re...",1 place de la Porte de Versailles,Paris,Paris expo Porte de Versailles,Île-de-France,2022-11-10,fr,Point,"[2.287499, 48.832488]",15e Arrondissement,09:30 - 19:30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,Sirba Octet &amp; Balalaïka,"[48.765358, 2.268554]",2022-06-07,NaN,Sirba Octet &amp; Balalaïka,15.0,SIRBALALAÏKA,http://openagenda.com/event/test-sirba-octet-a...,"Voir un concert du Sirba Octet, c’est comme pa...","254, avenue de la Division Leclerc 92290 châte...",Hauts-de-Seine,L'Azimut - Théâtre La Piscine,Île-de-France,2022-06-07,fr,Point,"[2.268554, 48.765358]",NaN,20:30 - 22:00
96,L'Aparté vous accueille ce samedi,"[48.824529, 2.274148]",2022-06-04,"CLAVIM,Aparté,Maison Verte",L'Aparté vous accueille ce samedi,NaN,Lieu d’accueil pour les bébés et les enfants d...,http://openagenda.com/event/laparte-vous-accue...,L’**Aparté** vous accueille sans rendez-vous c...,"60 rue du Général Leclerc, 92130 Issy-les-Moul...",Hauts-de-Seine,L'Aparté,Île-de-France,2022-06-04,fr,Point,"[2.274148, 48.824529]",NaN,15:00 - 18:30
97,Erstkommunion,"[48.870856, 2.285033]",2022-06-04,NaN,Erstkommunion,NaN,Festgottesdienst der Erstkommunion in St. Hono...,http://openagenda.com/event/erstkommunion-4029827,NaN,"66 bis avenue Raymond-Poincaré, 75116 Paris",Paris,St Honoré d'Eylau,Île-de-France,2022-06-04,fr,Point,"[2.285033, 48.870856]",Paris 16e Arrondissement,10:30 - 11:50
98,Solennité de Pentecôte,"[48.855518, 2.354704]",2022-06-04,NaN,Solennité de Pentecôte,NaN,Célébrer le don de l'Esprit Saint avec les Fra...,http://openagenda.com/event/solennite-de-pente...,* **Samedi 4 juin 2022**\n\n12h30 : messe du...,"13, rue des Barres, 75004 Paris",Paris,"Église Saint-Gervais,Paris",Île-de-France,2022-06-05,fr,Point,"[2.354704, 48.855518]",Paris 4e Arrondissement,12:30 - 13:00


In [3]:
database = pd.read_csv('events_database.zip', compression='zip', header=0, sep=';')

/opt/conda/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (21) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [52]:
database[['Description', 'Détails', 'Mots clés']].head(5)

,Description,Détails,Mots clés
0,Découvrir le poney dans un séjour multisports ...,L'UCPA propose ses vacances apprenantes : Un s...,"UCPA,sport,nature,SFJBOI31E,import-ucpa-20200630"
1,"Un séjour qui combine sport, collectivité et f...",Cette colo te permettra de découvrir les multi...,"jeux olympiques,sport,import-pep"
2,Le top de la glisse de la côte californienne,L'UCPA propose ses vacances apprenantes : Un s...,"UCPA,sport,nature,SFJPON34R,import-ucpa-20200630"
3,Compagnie AtmosphèreS - mise en scène : Sara V...,"Camille, Camille, Camille\n-------------------...","Camille Claudel,Rodin,Paul claudel,sculptrice,..."
4,Mise en scène par Agnès Braunschweig. Cie Pros...,Je reviens de la vérité de Charlotte Delbo. Mi...,"Résistance,Déportation,Deuxième guerre mondial..."


In [38]:
def NaN_to_String(dataframe):
    df_temp = dataframe.copy()
    df_temp[df_temp.isna()] = ""
    return df_temp.apply(lambda s: s.replace("\n", "").replace("\r", "") + ". ")

text_desc = (NaN_to_String(database['Description']) + NaN_to_String(database['Détails']) + NaN_to_String(database['Mots clés']).apply(lambda s: s.replace(",", ", ")))

In [56]:
a = text_desc.head(10000).sum()

In [55]:
text = text_desc.head(10000).sum().lower()
words = nltk.word_tokenize(text, language='french')
words = [word for word in words if word.isalpha()]
words = [w for w in words if not w in stop_words]
dic_words = Counter(words)
dic_words.most_common(25)

[('a', 4404),
 ('plus', 3203),
 ('paris', 2476),
 ('musique', 2439),
 ('cette', 2386),
 ('https', 2328),
 ('the', 2253),
 ('http', 1935),
 ('tout', 1624),
 ('entre', 1623),
 ('comme', 1473),
 ('scène', 1455),
 ('concert', 1438),
 ('monde', 1387),
 ('amp', 1378),
 ('ans', 1351),
 ('tous', 1351),
 ('public', 1306),
 ('théâtre', 1305),
 ('découvrir', 1288),
 ('deux', 1281),
 ('leurs', 1222),
 ('atelier', 1185),
 ('faire', 1111),
 ('aussi', 1087)]